One of the primary modules in `py-goldsberry` is the `player` module. It provides access to a multitude of player-level statistics. 

Each class in the `player` module requires a specific **playerID**. If you have looked through the first tutorial, you can see that `py-goldsberry` has a built-in function that makes it easy to find the **playerID**s for a given season.

In [1]:
import goldsberry
import pandas as pd
goldsberry.__version__

'0.6.0'

One of the many things you can do with `py-goldsberry` is generate a list of game logs for a single player or the entire league (depending on what you desire). This can be accomplished very easily using two built-in methods and a simple custom function.

First, we generate a list of players from the 2014 season using the built-in `PlayerList()` function, and convert it to a Pandas DataFrame.

In [2]:
players_2014 = goldsberry.PlayerList(2014)

In [4]:
players_2014 = pd.DataFrame(players_2014)

In [5]:
players_2014.sample(5)

,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,TO_YEAR
130,"Douglas-Roberts, Chris",2008,201604,chris_douglas-roberts,0,,,,0,,2014
414,"Smith, J.R.",2004,2747,jr_smith,1,CLE,Cleveland,cavaliers,1610612739,Cavaliers,2014
204,"Hayes, Charles",2005,101236,chuck_hayes,1,TOR,Toronto,raptors,1610612761,Raptors,2014
251,"Jones, Perry",2012,203103,perry_jones,1,OKC,Oklahoma City,thunder,1610612760,Thunder,2015
56,"Booker, Trevor",2010,202344,trevor_booker,1,UTA,Utah,jazz,1610612762,Jazz,2015


When you have the data into a DataFrame, you can take advantage of the Pandas functionality to search for specific players, teams, rookie cohorts, etc...

Let's start by looking for just James Harden.

In [6]:
players_2014.ix[players_2014['DISPLAY_LAST_COMMA_FIRST'].str.contains("Harden")]

,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,TO_YEAR
196,"Harden, James",2009,201935,james_harden,1,HOU,Houston,rockets,1610612745,Rockets,2015


Fortunately, there is only one player with `Harden` somewhere in his name. If we had searched for `James`, it would have been a bit of a different story.

Because we want to get information on James Harden, we need to make note of the value in his **PERSON_ID** column. This is the unique id number that is associated with Harden in the NBA database. Anytime we want to search for James Harden related information, this will be a value to remember. 

To make it easy to remember, I'm going to save it as a variable in our environment that we can call it anytime we want. It's a bit easier for me to remember `harden_id` than `201935`.

In [7]:
harden_id = '201935'

###Game Logs

One of many pieces of available data for a player is their game logs. You can access these by using the `goldsberry.player.game_logs()` class and passing in the playerID. 

There are a few variables that can be manipulated in the game_logs to adjust the data that gets returned. The most important is the `season` argument. You can pass in whatever season you are interested in using the first four digits of the season. For example, if you want all of the game logs from the 2014-15 season, you pass `season=2014` when you call the class. If you wanted game logs from the 2009-2010 season, you would pass `season=2009`.

In [29]:
harden_2014_logs = goldsberry.player.game_logs(harden_id, season = 2014)

Now that we've collected the data from the NBA website, we want to create a Pandas DataFrame to view an analyze. 

In [30]:
harden_2014_logs = pd.DataFrame(harden_2014_logs.logs())

Notice that we passed `harden_2014_logs.logs()` and not `harden_2014_logs` to the DataFrame constructor. This is because, with many of the calls, there is more than one set of data that is returned. Instead of making multiple calls to the NBA's server, a single call is made and then the class methods return different types of data. 

(Until documentation is complete, take advantage of the [TAB] complete feature in jupyter.)

In [10]:
harden_2014_logs.head()

,AST,BLK,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,...,PF,PLUS_MINUS,PTS,Player_ID,REB,SEASON_ID,STL,TOV,VIDEO_AVAILABLE,WL
0,10,1,11,5,2,0.400,8,3,0.375,8,...,2,28,16,201935,11,22014,1,6,1,W
1,6,2,4,8,1,0.125,20,7,0.350,16,...,1,0,29,201935,6,22014,1,2,1,W
2,7,1,1,6,1,0.167,18,10,0.556,9,...,2,1,30,201935,1,22014,1,4,1,W
3,10,0,2,5,2,0.400,19,5,0.263,6,...,2,-2,16,201935,2,22014,3,4,1,L
4,4,0,2,6,2,0.333,15,6,0.400,8,...,1,-26,22,201935,4,22014,1,1,1,L


Now that we know how to get a single players game logs, we can easily get game logs for a list of players by creating a simple function.

In [25]:
def get_all_logs(playerids, season=2014):
    logs = [] # Empty List to store results
    for i in playerids:
        try:
            i_log = goldsberry.player.game_logs(i, season=season)
            logs = logs + i_log.logs() # leveraging Goldsberry returning a list instead of a DataFrame
        except:
            ValueError("no record for " + str(i))
    return logs

In the function above, notice we do not construct any DataFrames. `py-goldsberry` returns lists, not DataFrames. I did this so that loops like the one above would run faster. There is no need to convert each players data into a DataFrame and merge when all observations can be placed in a single list and converted to a DataFrame upon completion.

There is a bit of error handling just in case a playerID is fed to the function that is not valid.

Before we execute our function, we need to create a list of playerids for which we want game logs. 

Once this list is created, we pass the list as well as the season into the function, saving the results as `all_logs`

In [27]:
playerids = players_2014['PERSON_ID'].tolist()
all_logs = get_all_logs(playerids, season=2014)

This will take a bit of time to execute (depending on the speed of your internet connection). Once it has completed, you can contruct a DataFrame by passing the results to the DataFrame constructor. Once you are done, you can sample your results.

In [31]:
all_logs_df = pd.DataFrame(all_logs)
all_logs_df.sample(10)

,AST,BLK,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,...,PF,PLUS_MINUS,PTS,Player_ID,REB,SEASON_ID,STL,TOV,VIDEO_AVAILABLE,WL
6960,3,0,2,2,0,0.000,9,3,0.333,2,...,4,-13,7,201609,2,22014,0,1,1,L
7194,2,3,13,0,0,0.000,12,5,0.417,4,...,3,1,12,1495,14,22014,2,4,1,W
22134,1,0,0,3,1,0.333,4,1,0.250,0,...,1,-1,3,203503,1,22014,0,0,1,L
16361,0,2,3,0,0,0.000,3,1,0.333,2,...,1,-9,2,201580,3,22014,0,1,1,L
1049,2,2,9,0,0,0.000,7,3,0.429,3,...,3,-1,7,203507,9,22014,0,3,1,W
1831,0,0,3,1,1,1.000,6,5,0.833,2,...,1,10,13,203084,3,22014,0,1,1,W
12055,2,1,6,5,2,0.400,13,4,0.308,4,...,4,-4,14,201586,7,22014,0,3,1,L
724,2,2,2,0,0,0.000,3,1,0.333,1,...,4,-7,2,200811,4,22014,0,0,1,L
10304,7,1,1,6,1,0.167,18,10,0.556,9,...,2,1,30,201935,1,22014,1,4,1,W
10076,1,0,3,6,3,0.500,12,6,0.500,0,...,1,-4,15,203798,4,22014,1,1,1,W


Once you have your list of game logs, you might want to add some additional player information to enrich the dataset. Take advantage of the Pandas merge feature to merge the `players_2014` and `all_logs_df` dataframes together

In [32]:
player_game_logs_2014_15 = pd.merge(players_2014, all_logs_df, left_on="PERSON_ID", right_on="Player_ID", how="left")

In [33]:
player_game_logs_2014_15.sample(10)

,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,...,PF,PLUS_MINUS,PTS,Player_ID,REB,SEASON_ID,STL,TOV,VIDEO_AVAILABLE,WL
16175,"McCollum, CJ",2013,203468,cj_mccollum,1,POR,Portland,blazers,1610612757,Trail Blazers,...,3,4,4,203468,0,22014,0,1,1,L
13256,"Johnson, Wesley",2010,202325,wesley_johnson,1,LAL,Los Angeles,lakers,1610612747,Lakers,...,1,-9,9,202325,3,22014,1,3,1,L
9914,"Green, Jeff",2007,201145,jeff_green,1,MEM,Memphis,grizzlies,1610612763,Grizzlies,...,1,-9,15,201145,7,22014,0,3,1,W
25050,"Williams, Deron",2005,101114,deron_williams,1,BKN,Brooklyn,nets,1610612751,Nets,...,2,1,13,101114,6,22014,1,0,1,L
10879,"Hayward, Gordon",2010,202330,gordon_hayward,1,UTA,Utah,jazz,1610612762,Jazz,...,1,-17,17,202330,1,22014,1,2,1,L
8372,"Frye, Channing",2005,101112,channing_frye,1,ORL,Orlando,magic,1610612753,Magic,...,0,-16,8,101112,2,22014,1,1,1,L
25331,"Williams, Mo",2003,2590,maurice_williams,1,CHA,Charlotte,hornets,1610612766,Hornets,...,4,24,13,2590,2,22014,1,1,1,W
9815,"Green, Gerald",2005,101123,gerald_green,1,PHX,Phoenix,suns,1610612756,Suns,...,2,15,10,101123,5,22014,0,0,1,W
12147,"Ilyasova, Ersan",2006,101141,ersan_ilyasova,1,MIL,Milwaukee,bucks,1610612749,Bucks,...,3,-2,16,101141,4,22014,2,2,1,W
17922,"Nelson, Jameer",2004,2749,jameer_nelson,1,DEN,Denver,nuggets,1610612743,Nuggets,...,1,3,9,2749,0,22014,1,0,1,L
